In [2]:
import torch

In [23]:
a = torch.tensor([[1,0], [1, 1], [0, 0], [1, 0], [0, 1], [0, 0]])
unique = torch.unique(a, dim=0)
H = torch.all(a[:, None] == unique, dim=-1)
w = torch.tensor([0.8, 0.1, 0.1, 0.01])
doubles = torch.where(abs(torch.sum(a[:, None] - unique, dim=-1)) == 2)
print(abs(torch.sum(a[:, None] - unique, dim=-1)) == 2)
doubles

tensor([[False, False, False, False],
        [ True, False, False, False],
        [False, False, False,  True],
        [False, False, False, False],
        [False, False, False, False],
        [False, False, False,  True]])


(tensor([1, 2, 5]), tensor([0, 3, 3]))

In [24]:
def lipkin_local(dist_s, unique_s, weight_s, eps, V, W):
    size = weight_s[torch.where(torch.all(dist_s[:, None] == unique_s, dim=-1))[1]]
    H_0 = eps*(torch.sum(dist_s == 1, dim=1) - torch.sum(dist_s == 0, dim=1))
    size_mask, weight_mask = torch.where(torch.abs(torch.sum(dist_s[:, None]-unique_s, dim=-1)) == 2)
    H_1 = torch.zeros_like(size)
    H_1[size_mask] = V*weight_s[weight_mask]
    sum = 0.5*H_0 - H_1
    return sum

lipkin_local(a, unique, w, 1, 0.5, 0)

tensor([ 0.0000,  0.6000, -1.0050,  0.0000,  0.0000, -1.0050])

In [28]:
import itertools
lst = list(itertools.product([0, 1], repeat=3))
lst

[(0, 0, 0),
 (0, 0, 1),
 (0, 1, 0),
 (0, 1, 1),
 (1, 0, 0),
 (1, 0, 1),
 (1, 1, 0),
 (1, 1, 1)]

In [407]:
dist_s = torch.tensor([[0, 0]]*24 + [[1, 1]]*4)
size = dist_s.shape[0]
unique, weight = torch.unique(dist_s, dim=0, return_counts=True)
weight = torch.sqrt(weight/size)
mask_u = torch.where(torch.all(dist_s[:, None] == unique, dim = -1))[1]

H_0 = 0.5*(torch.sum(unique == 1, dim=-1) - torch.sum(unique == 0, dim = -1))
H_1 = torch.sum(weight[:, None]*(abs(torch.sum((unique[:, None] - unique), dim=-1)) == 2), dim=0)/weight
E = (H_0[mask_u] - H_1[mask_u])
torch.mean(E)

tensor(-1.4141)

In [399]:
d = torch.tensor([[1, 1, 1], [1, 1, 1], [1, 1, 1], [1, 1, 1]])
a = torch.tensor([[1, 1], [1, 0], [0, 1], [0, 0]])
b = torch.tensor([2, 2, 2])

a[:, :, None]@d[:, None, :]



tensor([[[1, 1, 1],
         [1, 1, 1]],

        [[1, 1, 1],
         [0, 0, 0]],

        [[0, 0, 0],
         [1, 1, 1]],

        [[0, 0, 0],
         [0, 0, 0]]])

In [ ]:
import netket as nk
from netket.operator.spin import sigmax, sigmaz

In [ ]:
def ising_true(N, J, L):
    hi = nk.hilbert.Spin(s=1/2, N=N) 
    H = sum([L*sigmax(hi,i) for i in range(N)])
    H += sum([J*sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
    sp_h=H.to_sparse()
    from scipy.sparse.linalg import eigsh
    eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")
    E_gs = eig_vals[0]
    return E_gs

In [410]:
dist_s = torch.tensor([[1, 1], [1, 0], [0, 1], [1, 0], [0, 1], [0,0], [0, 0], [0, 1], [0, 0], [0, 0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [1,1], [1,1], [1,1], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0], [0,0]])
size = dist_s.shape[0]
unique, weight = torch.unique(dist_s, dim=0, return_counts=True)
weight = torch.sqrt(weight/size)
mask_u = torch.where(torch.all(dist_s[:, None] == unique, dim = -1))[1]

unique = 2*unique-1
H = -torch.sum(torch.roll(unique, 1, dims=-1)*unique, dim = -1)
H

tensor([-2,  2,  2, -2])